In [5]:
import leafmap
import rasterio
import numpy as np
from sklearn.cluster import KMeans

In [7]:
# Load data
def load_image(year):
    folder = f'data/sentinel/{year}'
    bands = [rasterio.open(f"{folder}/B0{b}.jp2").read(1) for b in [2,3,4,8]]
    return np.stack(bands, axis=-1)

def load_predictor(path):
    with rasterio.open(path) as src:
        return src.read(1)

In [9]:
# Calculate NDVI
def calc_ndvi(img):
    return (img[:,:,3] - img[:,:,2]) / (img[:,:,3] + img[:,:,2] + 1e-6)

# Calculate EVI
def calc_evi(img):
    # EVI = 2.5 * (NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1)
    nir = img[:, :, 3]   # B08
    red = img[:, :, 2]   # B04
    blue = img[:, :, 0]  # B02
    return 2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1.0 + 1e-6)

In [13]:
# KMeans Classification
def classify_kmeans(img):
    data = img[:,:, [2,3]].reshape(-1,2)
    km = KMeans(2).fit(data)
    return km.labels_.reshape(img.shape[0], img.shape[1])

In [15]:
# Display NDVI in map
def display_map(arr, name='NDVI'):
    import leafmap
    import rasterio
    from rasterio.transform import from_origin

    # Simulate a georeference transform (adjust to your actual bounds/resolution!)
    transform = from_origin(-69, -15, 0.0001, 0.0001)  # (west, north, xres, yres)
    height, width = arr.shape
    center_row = height // 2
    center_col = width // 2
    center_lon, center_lat = transform * (center_col, center_row)

    coords = []
    for row in range(0, arr.shape[0], 10):  # subsample for performance
        for col in range(0, arr.shape[1], 10):
            val = arr[row, col]
            if not np.isnan(val):
                lon, lat = transform * (col, row)
                coords.append([lat, lon, float(val)])

    # Convert to DataFrame
    import pandas as pd
    df = pd.DataFrame(coords, columns=["latitude", "longitude", "value"])

    # Create and show the map
    m = leafmap.Map(center=[center_lat, center_lon], zoom=7)
    m.add_heatmap(data=df, latitude="latitude", longitude="longitude", value="value", name=name, radius=10)
    m.add_layer_control()
    display(m)


In [17]:
data = {
    year: {
        'sentinel': load_image(year),
        'pop': load_predictor(f"data/population/{year}.tif"),
        'precip': load_predictor(f"data/precipitation/{year}.tif"),
        'temp': load_predictor(f"data/temperature/{year}.tif")
    } for year in range(2020, 2025)
}
dem = load_predictor("data/dem.tif")

In [19]:
import ipywidgets as widgets
from IPython.display import display

In [21]:
year_dropdown = widgets.Dropdown(options=[2020, 2021, 2022, 2023, 2024], description='Year:')
index_select = widgets.Dropdown(options=['NDVI', 'EVI', 'Classification'], description='Analysis:')
run_button = widgets.Button(description='Run')
output = widgets.Output()

In [27]:
ui = widgets.VBox([year_dropdown, index_select, run_button, output])
display(ui)

In [25]:
def on_run_clicked(b):
    output.clear_output()
    year = year_dropdown.value
    analysis = index_select.value
    img = data[year]['sentinel']
    
    with output:
        if analysis == 'NDVI':
            ndvi = calc_ndvi(img)
            display_map(ndvi, name=f"NDVI {year}")
        elif analysis == 'EVI':
            evi = calc_evi(img)
            display_map(evi, name=f"EVI {year}")
        elif analysis == 'Classification':
            cls = classify_kmeans(img)
            display_map(cls.astype(float), name=f"Land Cover {year}")

run_button.on_click(on_run_clicked)